# Importing the required modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string

from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

# Loading and Preprocessing the dataset

In [9]:
class Dataset:
   def __init__(self, data_dir : str):
       self.dir = data_dir
       self.headlines = []
       self.df = None
       self.text = None
       self.tokenizer = Tokenizer()
       self.input = []
       self.max_length = 0
       self.words = None
       self.total_words = 0
       self.x_train = None
       self.y_train = None


   def load_dataset(self):
       self.df = pd.read_csv(self.dir)
       self.headlines = list(self.df.headline.values)
       self.headlines = [h for h in self.headlines if h != "Unknown"]

   def __clean_text(self,text : str):
       self.ch = []
       for i in text:
           if i not in string.punctuation:
              self.ch.append(i)
       self.text = " ".join(self.ch)
       preprocessed_data = self.text.encode("utf8").decode("ascii",'ignore')
       return preprocessed_data

   def __create_word_vector(self, preprocessed_data : list[str]):
       self.tokenizer.fit_on_texts(preprocessed_data)
       self.total_words = len(self.tokenizer.word_index) + 1

       for line in preprocessed_data:
            tokens = self.tokenizer.texts_to_sequences([line])[0]
       for i in range(1, len(tokens)):
            n_gram_sequence = tokens[:i+1]
            self.input.append(n_gram_sequence)

   def __pad_vectors(self):
       self.max_length = max(map(len, self.input))
       self.input = np.array(pad_sequences(self.input, 
                                  maxlen=self.max_length, 
                                  padding='pre'))
    
       self.x_train, self.y_train = self.input[:,:-1],self.input[:,-1]
       self.y_train = ku.to_categorical(self.y_train, 
                                 num_classes=self.total_words)

   def preprocess_data(self):
       self.headlines = list(map(self.__clean_text, 
                                 self.headlines))
       self.__create_word_vector(self.headlines)
       self.__pad_vectors()


   def summarise_data(self):
       pass

In [10]:
d = Dataset("articles\ArticlesApril2018.csv")

In [11]:
d.load_dataset()

In [12]:
d.headlines

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafaring Past, and Muddied Its Future',
 'A Quote Disproved',
 'Hot Stuff Turns Cold',
 'At the Top,

In [13]:
d.preprocess_data()

In [14]:
d.input

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  3, 10,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  3, 10,  1,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  3, 10,  1,  7,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         3, 10,  1,  7,  1,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,
        10,  1,  7,  1,  6,  5],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 10,
         1,  7,  1,  6,  5,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 10,  1,
         7,  1,  6,  5,  8,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 10,  1,  7,
         1,  6,  5,  8,  4,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,

# Design model for Text Generator

In [31]:
class TextGenerator:
    def __init__(self, max_len : int, n_words : int, x_train, y_train):
        self.input_dim = max_len - 1
        self.model = None
        self.nwords = n_words   
        self.x_train = x_train
        self.y_train = y_train

    def design_model(self):
        self.model = Sequential()
        self.model.add(Embedding(self.nwords, 
                                 10, 
                                 input_length=self.input_dim))
        self.model.add(LSTM(100))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(self.nwords, 
                             activation='softmax'))
        self.model.summary()

    def compile_model(self):
        self.model.compile(loss='categorical_crossentropy', 
                           optimizer='adam')

    def fit_model(self):
        self.model.fit(self.x_train, self.y_train, epochs=100)

    def generate_text(self):
        pass

In [32]:
tg = TextGenerator(d.max_length, d.total_words, d.x_train, d.y_train)

In [33]:
tg.design_model()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 21, 10)            370       
                                                                 
 lstm_2 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 37)                3737      
                                                                 
Total params: 48,507
Trainable params: 48,507
Non-trainable params: 0
_________________________________________________________________


In [34]:
tg.compile_model()

In [35]:
tg.fit_model()

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 3.6111
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 3.6052
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 3.5990
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 3.5921
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 3.5836
Epoch 6/100
1/1 [==============================] - 0s 20ms/step - loss: 3.5729
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 3.5607
Epoch 8/100
1/1 [==============================] - 0s 24ms/step - loss: 3.5464
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 3.5283
Epoch 10/100
1/1 [==============================] - 0s 25ms/step - loss: 3.4962
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - loss: 3.4615
Epoch 12/100
1/1 [==============================] - 0s 35ms/step - loss: 3.4110
Epoch 13/100
1/1 [==============================] -